# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 66 new papers today
          29 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/28 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.10448


extracting tarball to tmp_2211.10448...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.10449


extracting tarball to tmp_2211.10449...

...

extracting tarball to tmp_2211.11408...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.11431


extracting tarball to tmp_2211.11431... done.
Retrieving document from  https://arxiv.org/e-print/2211.11440


extracting tarball to tmp_2211.11440... done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.11440/jhepexample.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10490-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10490) | **Direct Evidence of Photochemistry in an Exoplanet Atmosphere**  |
|| Shang-Min Tsai, et al. -- incl., <mark>Elspeth K. H. Lee</mark>, <mark>Xi Zhang</mark>, <mark>Lili Alderson</mark>, <mark>Jayesh M Goyal</mark>, <mark>Laura Kreidberg</mark>, <mark>Evert Nasedkin</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *40 pages, 13 figures, under review at Nature*|
|**Abstract**| Photochemistry is a fundamental process of planetary atmospheres that is integral to habitability, atmospheric composition and stability, and aerosol formation. However, no unambiguous photochemical products have been detected in exoplanet atmospheres to date. Here we show that photochemically produced sulphur dioxide (SO$_2$) is present in the atmosphere of the hot, giant exoplanet WASP-39b, as constrained by data from the JWST Transiting Exoplanet Early Release Science Program and informed by a suite of photochemical models. We find that SO$_2$ is produced by successive oxidation of sulphur radicals freed when hydrogen sulphide (H$_2$S) is destroyed. The SO$_2$ distribution computed by the photochemical models robustly explains the 4.05 $\mu$m spectral feature seen in JWST transmission spectra [Rustamkulov et al.(submitted), Alderson et al.(submitted)] and leads to observable features at ultraviolet and thermal infrared wavelengths not available from the current observations. The sensitivity of the SO$_2$ feature to the enrichment of heavy elements in the atmosphere ("metallicity") suggests that it can be used as a powerful tracer of atmospheric properties, with our results implying a metallicity of $\sim$10$\times$ solar for WASP-39b. Through providing improved constraints on bulk metallicity and sulphur abundance, the detection of SO$_2$ opens a new avenue for the investigation of giant-planet formation. Our work demonstrates that sulphur photochemistry may be readily observable for exoplanets with super-solar metallicity and equilibrium temperatures $\gtrsim$750 K. The confirmation of photochemistry through the agreement between theoretical predictions and observational data is pivotal for further atmospheric characterisation studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10448-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10448) | **A General Bayesian Framework to Account for Foreground Map Errors in  Global 21-cm Experiments**  |
|| Michael Pagano, Peter Sims, <mark>Adrian Liu</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *18 pages, 13 figures*|
|**Abstract**| Measurement of the global 21-cm signal during Cosmic Dawn (CD) and the Epoch of Reionization (EoR) is made difficult by bright foreground emission which is 2-5 orders of magnitude larger than the expected signal. Fitting for a physics-motivated parametric forward model of the data within a Bayesian framework provides a robust means to separate the signal from the foregrounds, given sufficient information about the instrument and sky. It has previously been demonstrated that, within such a modelling framework, a foreground model of sufficient fidelity can be generated by dividing the sky into $N$ regions and scaling a base map assuming a distinct uniform spectral index in each region. Using the Radio Experiment for the Analysis of Cosmic Hydrogen (REACH) as our fiducial instrument, we show that, if unaccounted-for, amplitude errors in low-frequency radio maps used for our base map model will prevent recovery of the 21-cm signal within this framework, and that the level of bias in the recovered 21-cm signal is proportional to the amplitude and the correlation length of the base-map errors in the region. We introduce an updated foreground model that is capable of accounting for these measurement errors by fitting for a monopole offset and a set of spatially-dependent scale factors describing the ratio of the true and model sky temperatures, with the size of the set determined by Bayesian evidence-based model comparison. We show that our model is flexible enough to account for multiple foreground error scenarios allowing the 21-cm sky-averaged signal to be detected without bias from simulated observations with a smooth conical log spiral antenna. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10449-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10449) | **The Molecular-Gas Main Sequence and Schmidt-Kennicutt relation are  fundamental, the Star-Forming Main Sequence is a (useful) byproduct**  |
|| William M. Baker, et al. -- incl., <mark>Lihwai Lin</mark>, <mark>Mallory Thorp</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *16 pages, 14 figures, Accepted for publication in MNRAS*|
|**Abstract**| We investigate the relationship between the star formation rate (SFR), stellar mass ($M_*$) and molecular gas mass ($M_{H_2}$) for local star-forming galaxies. We further investigate these relationships for high-z (z=1-3) galaxies and for the hosts of a local sample of Active Galactic Nuclei (AGN). We explore which of these dependencies are intrinsic and which are an indirect by-product by employing partial correlation coefficients and random forest regression. We find that for local star-forming galaxies, high-z galaxies, and AGN host galaxies, the Schmidt-Kennicutt relation (SK, between $M_{H_2}$ and SFR), and the Molecular Gas Main Sequence (MGMS, between $M_{H_2}$ and $M_*$) are intrinsic primary relations, while the relationship between $M_*$ and SFR, i.e. the Star-Forming Main Sequence (SFMS), is an indirect by-product of the former two. Hence the Star-Forming Main Sequence is not a fundamental scaling relation for local or high-redshift galaxies. We find evidence for both the evolution of the MGMS and SK relation over cosmic time, where, at a given stellar mass, the higher the redshift, the greater the molecular gas mass and the star formation efficiency. We offer a parameterisation of both the MGMS and SK relation's evolution with redshift, showing how they combine to form the observed evolution of the SFMS. In addition, we find that the local AGN host galaxies follow an AGN-MGMS relation (as well as a AGN-SK relation), where the MGMS is offset to lower $M_{H_2}$ for a given $M_*$ compared to local SF galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.11440-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.11440) | **Kinetically stabilized inflation**  |
|| <mark>Changhong Li</mark>, Hao Gong, Yeuk-kwan Edna Cheung |
|*Appeared on*| *2022-11-22*|
|*Comments*| *16 pages, 4 figures*|
|**Abstract**| In this work, we propose a string-inspired two fields inflation model to address the fine-tuning problem that the standard inflation model suffers. The fast-rolling tachyon $\mathcal{T}$ originated from the D-brane and anti-D-brane pair annihilation locks the inflaton $\varphi$ slowly rolling on a Higgs-like potential $V(\varphi)=-m_\varphi^2\varphi^2+\lambda \varphi^4$ and drives a kinetically stabilized (KS) inflation. Our numerical simulation confirms such a solution is a dynamic attractor. In particular, for $\lambda< 0.8\times 10^{-3}$, the e-folding number contributed by the KS inflation phase can be larger than $62$ to solve the horizon and flatness problems of Big Bang theory. Notably, this KS inflation generates a nearly scale-invariant primordial curvature perturbations spectrum consistent with current cosmic microwave background (CMB) observations. It predicts a low tensor-to-scalar ratio, which the current primordial gravitational wave background (the B-modes in CMB) searches favor. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10487-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10487) | **Early Release Science of the exoplanet WASP-39b with JWST NIRSpec PRISM**  |
|| Z. Rustamkulov, et al. -- incl., <mark>M. R. Line</mark>, <mark>J. M. Goyal</mark>, <mark>L. Kreidberg</mark>, <mark>T. Mikal-Evans</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *41 pages, 4 main figures, 10 extended data figures, 4 tables. Under review in Nature*|
|**Abstract**| Transmission spectroscopy of exoplanets has revealed signatures of water vapor, aerosols, and alkali metals in a few dozen exoplanet atmospheres. However, these previous inferences with the Hubble and Spitzer Space Telescopes were hindered by the observations' relatively narrow wavelength range and spectral resolving power, which precluded the unambiguous identification of other chemical species$-$in particular the primary carbon-bearing molecules. Here we report a broad-wavelength 0.5-5.5 $\mu$m atmospheric transmission spectrum of WASP-39 b, a 1200 K, roughly Saturn-mass, Jupiter-radius exoplanet, measured with JWST NIRSpec's PRISM mode as part of the JWST Transiting Exoplanet Community Early Release Science Team program. We robustly detect multiple chemical species at high significance, including Na (19$\sigma$), H$_2$O (33$\sigma$), CO$_2$ (28$\sigma$), and CO (7$\sigma$). The non-detection of CH$_4$, combined with a strong CO$_2$ feature, favours atmospheric models with a super-solar atmospheric metallicity. An unanticipated absorption feature at 4$\mu$m is best explained by SO$_2$ (2.7$\sigma$), which could be a tracer of atmospheric photochemistry. These observations demonstrate JWST's sensitivity to a rich diversity of exoplanet compositions and chemical processes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10488-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10488) | **Early Release Science of the Exoplanet WASP-39b with JWST NIRSpec G395H**  |
|| <mark>Lili Alderson</mark>, et al. -- incl., <mark>Jayesh M. Goyal</mark>, <mark>Elspeth K. H. Lee</mark>, <mark>Thomas Mikal-Evans</mark>, <mark>Lakeisha Ramos-Rosado</mark>, <mark>Laura Kreidberg</mark>, <mark>Michael R. Line</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *44 pages, 11 figures, 3 tables. Resubmitted after revision to Nature*|
|**Abstract**| Measuring the abundances of carbon and oxygen in exoplanet atmospheres is considered a crucial avenue for unlocking the formation and evolution of exoplanetary systems. Access to an exoplanet's chemical inventory requires high-precision observations, often inferred from individual molecular detections with low-resolution space-based and high-resolution ground-based facilities. Here we report the medium-resolution (R$\sim$600) transmission spectrum of an exoplanet atmosphere between 3-5 $\mu$m covering multiple absorption features for the Saturn-mass exoplanet WASP-39b, obtained with JWST NIRSpec G395H. Our observations achieve 1.46x photon precision, providing an average transit depth uncertainty of 221 ppm per spectroscopic bin, and present minimal impacts from systematic effects. We detect significant absorption from CO$_2$ (28.5$\sigma$) and H$_2$O (21.5$\sigma$), and identify SO$_2$ as the source of absorption at 4.1 $\mu$m (4.8$\sigma$). Best-fit atmospheric models range between 3 and 10x solar metallicity, with sub-solar to solar C/O ratios. These results, including the detection of SO$_2$, underscore the importance of characterising the chemistry in exoplanet atmospheres, and showcase NIRSpec G395H as an excellent mode for time series observations over this critical wavelength range. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10489-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10489) | **Early Release Science of the exoplanet WASP-39b with JWST NIRCam**  |
|| Eva-Maria Ahrer, et al. -- incl., <mark>Sebastian Zieba</mark>, <mark>Jayesh M Goyal</mark>, <mark>Joseph Harrington</mark>, <mark>Laura Kreidberg</mark>, <mark>Nikole K. Lewis</mark>, <mark>Michael R. Line</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *35 pages, 13 figures, 3 tables, Nature, accepted*|
|**Abstract**| Measuring the metallicity and carbon-to-oxygen (C/O) ratio in exoplanet atmospheres is a fundamental step towards constraining the dominant chemical processes at work and, if in equilibrium, revealing planet formation histories. Transmission spectroscopy provides the necessary means by constraining the abundances of oxygen- and carbon-bearing species; however, this requires broad wavelength coverage, moderate spectral resolution, and high precision that, together, are not achievable with previous observatories. Now that JWST has commenced science operations, we are able to observe exoplanets at previously uncharted wavelengths and spectral resolutions. Here we report time-series observations of the transiting exoplanet WASP-39b using JWST's Near InfraRed Camera (NIRCam). The long-wavelength spectroscopic and short-wavelength photometric light curves span 2.0 - 4.0 $\mu$m, exhibit minimal systematics, and reveal well-defined molecular absorption features in the planet's spectrum. Specifically, we detect gaseous H$_2$O in the atmosphere and place an upper limit on the abundance of CH$_4$. The otherwise prominent CO$_2$ feature at 2.8 $\mu$m is largely masked by H$_2$O. The best-fit chemical equilibrium models favour an atmospheric metallicity of 1-100$\times$ solar (i.e., an enrichment of elements heavier than helium relative to the Sun) and a sub-stellar carbon-to-oxygen (C/O) ratio. The inferred high metallicity and low C/O ratio may indicate significant accretion of solid materials during planet formation or disequilibrium processes in the upper atmosphere. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10493-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10493) | **Early Release Science of the exoplanet WASP-39b with JWST NIRISS**  |
|| Adina D. Feinstein, et al. -- incl., <mark>Michael R. Line</mark>, <mark>Heather A. Knutson</mark>, <mark>Thomas Mikal-Evans</mark>, <mark>Jayesh M Goyal</mark>, <mark>Laura Kreidberg</mark> |
|*Appeared on*| *2022-11-22*|
|*Comments*| *48 pages, 12 figures, 2 tables. Under review at Nature*|
|**Abstract**| Transmission spectroscopy provides insight into the atmospheric properties and consequently the formation history, physics, and chemistry of transiting exoplanets. However, obtaining precise inferences of atmospheric properties from transmission spectra requires simultaneously measuring the strength and shape of multiple spectral absorption features from a wide range of chemical species. This has been challenging given the precision and wavelength coverage of previous observatories. Here, we present the transmission spectrum of the Saturn-mass exoplanet WASP-39b obtained using the SOSS mode of the NIRISS instrument on the JWST. This spectrum spans $0.6 - 2.8 \mu$m in wavelength and reveals multiple water absorption bands, the potassium resonance doublet, as well as signatures of clouds. The precision and broad wavelength coverage of NIRISS-SOSS allows us to break model degeneracies between cloud properties and the atmospheric composition of WASP-39b, favoring a heavy element enhancement ("metallicity") of $\sim 10 - 30 \times$ the solar value, a sub-solar carbon-to-oxygen (C/O) ratio, and a solar-to-super-solar potassium-to-oxygen (K/O) ratio. The observations are best explained by wavelength-dependent, non-gray clouds with inhomogeneous coverage of the planet's terminator. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

95  publications in the last 7 days.
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers